### Lab:  Model Validation With Gradient Boosting

Welcome to this evening's lab!  It's going to be a fun one.  For today's class, we're going to try and take a crack at model building in a wholistic way.  

Specifically, we're going to try and do three different things:

 - Try out different versions of our data, and use our validation scores to see if something was an improvement or not
 - We're going to adjust model parameters to try and adjust our results to help curb overfitting
 - We're going to try and find model parameters that maximize our score for our dataset
 
The idea is that we'll be able to do a mini-walkthrough to test what it's like to build and validate a model and try and improve our results.

**Step 1:** Using the suggestions from the homework prompt given previously, try and add 3-4 different features ( columns ) to your data, and use your validation score to determine if they improved your results.  

This is meant to be open ended, and to allow you a chance to re-discover material from previous labs.

In [2]:
# your code here
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline

df = pd.read_csv('../../data/restaurants.csv', parse_dates=['visit_date'])
df.drop('calendar_date', axis=1, inplace=True)

In [3]:
# define some functions that we can reuse
def create_val_splits(df, val_units=15, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    df = df.drop('visit_date', axis=1)
    train = df.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    
    if return_val:
        val   = train.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test
    
def denote_null_values(df):
    """Denotes whether or not there are null values or not"""
    empty_cols_query = df.isnull().sum() > 0
    empty_df_cols = df.loc[:, empty_cols_query].columns.tolist()
    for col in empty_df_cols:
        col_name = f"{col}_missing"
        df[col_name] = pd.isnull(df[col])
    return df

In [16]:
# we'll start of with an initial model fitting
# fill in missing values
df = denote_null_values(df)
df = df.fillna(0)

# and do an initial fitting
train, val, test = create_val_splits(df, return_val=True)

# create a pipeline, and get our model score
pipe = make_pipeline(ce.TargetEncoder(), GradientBoostingRegressor())

# split into X & y
X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_val, y_val = val.drop('visitors', axis=1), val['visitors']

# fit & score
pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.4587706012625258

This is our current baseline to work with.  Now we'll add some features.

In [20]:
# we'll do month & previous values, like we had before
# sort by date -- just to make sure
df.sort_values(by=['id', 'visit_date'], ascending=True, inplace=True)
df['month'] = df['visit_date'].dt.month
df['yesterday'] = df.groupby('id').apply(lambda x: x['visitors'].shift()).values

# denote the missing values, and fill
df = denote_null_values(df)
df = df.bfill()

# split the data, fit & score
train, val, test = create_val_splits(df, return_val=True)
X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_val, y_val = val.drop('visitors', axis=1), val['visitors']

pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.5162597129609126

That looks better.  Let's try doing this a few more times to see if we can keep making improvements.  Let's try our hand at some summary statistics -- comparing observations to some larger category.

In [4]:
# let's try creating a grouping that captures typical attendance for a city + day of week
# we'll first create training, validation, & test sets
df['city'] = df.area.str.split('-').str[0]
train, val, test = create_val_splits(df, return_val=True)

# and get our grouping -- notice we are doing this on the TEST set, and applying this to the validation set
city_avgs = train.groupby(['city', 'genre', 'day_of_week'])[['visitors']].mean().rename({'visitors': 'city_genre_day_visits'}, axis=1)

In [39]:
# and merge it back in
train = train.merge(city_avgs, left_on=['city', 'genre', 'day_of_week'], right_index=True, how='left')
val   = val.merge(city_avgs, left_on=['city', 'genre', 'day_of_week'], right_index=True, how='left')

# create our splits
X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_val, y_val = val.drop('visitors', axis=1), val['visitors']

# re-initialize the pipe to include the city column
target_encoder = ce.TargetEncoder()
pipe = make_pipeline(target_encoder, GradientBoostingRegressor())

# and fit + score
pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.531904270570581

Still getting better, so we'll keep things.  Notice that we haven't added this column to the entire dataset yet.......let's add it back into the entire dataset.

In [40]:
# merge things back into the entire dataset so it's a permanent column
df = df.merge(city_avgs, left_on=['city', 'genre', 'day_of_week'], right_index=True, how='left')

Now let's try adding in some moving average values to see if this helps.

In [9]:
df['visitors'].head(3)

0    25
1    32
2    29
Name: visitors, dtype: int64

In [13]:
pipe = make_pipeline(ce.TargetEncoder(), GradientBoostingRegressor())

In [19]:
genre_vals = df.groupby(['genre', 'day_of_week'])[['visitors']].mean().rename({'visitors': 'genre_mean'}, axis=1)

In [21]:
df2 = df.copy()
df2 = df2.merge(genre_vals, left_on=['genre', 'day_of_week'], right_index=True, how='left')

In [24]:
df2['diff'] = (df2['visitors'] - df2['genre_mean']) / df2['genre_mean']

In [27]:
df2 = df2.fillna(0)

In [32]:
df2['genre_mean'] = df2.groupby('id').apply(lambda x: x['genre_mean'].shift()).values

In [34]:
df2 = df2.bfill()

In [36]:
pipe.score(df2.drop(['visitors', 'visit_date'], axis=1), df2['visitors'])

0.9873638200164768

In [25]:
df2.head()

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,city,genre_mean,diff
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō,16.984222,0.471954
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō,17.049499,0.876888
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō,21.703760,0.336174
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō,24.718941,-0.109994
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō,14.543439,-0.587443


In [11]:
df.head()

,id,visit_date,visitors,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,city
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,Tōkyō


In [45]:
# we'll create window statistics for 7, 30, & 60 days -- notice the use of shift() here
df['7DayAvg']  = df.groupby('id').apply(lambda x: x['visitors'].rolling(7).mean().shift()).values
df['30DayAvg'] = df.groupby('id').apply(lambda x: x['visitors'].rolling(30).mean().shift()).values
df['60DayAvg'] = df.groupby('id').apply(lambda x: x['visitors'].rolling(60).mean().shift()).values

In [47]:
# note missing vals, backfill, and split the data
df = denote_null_values(df)
df = df.bfill()

train, val, test = create_val_splits(df, return_val=True)

X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_val, y_val     = val.drop('visitors', axis=1), val['visitors']

# fit and score
pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.556299886924623

So far, so good......all of these additions made modest, but real improvements to our out-of-sample scores, so we'll keep them.

**Step 2:** Try and reduce overfitting in your model, if it's persistent.  Ideally, you want your in-sample and out-of-sample scores to be about the same, or at least increasing or decreasing in proportional amounts.  

The idea here is two-fold:  see if you can narrow the gap between in-sample and out-of-sample results (using training & validation sets), while simultaneously **not** decreasing your model scores (or at least not by very much).  The idea being that the closer these two are, the more reliable your results are likely to be.

Some knobs you can turn:
 - `min_samples_leaf`: parameter in the category encoder that determines what cutoff point you can use for using the local vs. global average for the category
 - `subsample`: parameter in gbm that determines what fraction of your dataset to use at each boosting round.  This both reduces training time and makes each fitting round less related to the other
 - `max_features`: what portion of columns to use at each split.  This is very similar in purpose to `subsample`, but randomizes data at each split, vs. each round.

In [39]:
encoder = ce.TargetEncoder().set_params(min_samples_leaf=20)

In [41]:
df['visitors'].mean()

20.973761245180636

In [40]:
encoder.get_params()

{'cols': None,
 'drop_invariant': False,
 'handle_missing': 'value',
 'handle_unknown': 'value',
 'min_samples_leaf': 20,
 'return_df': True,
 'smoothing': 1.0,
 'verbose': 0}

In [48]:
# your code here
# we'll first see how our model is overfitting
pipe.score(X_train, y_train), pipe.score(X_val, y_val)

(0.5201554721545789, 0.556299886924623)

These results......are good.  Our out-sample-results are better than in-sample, so we won't complain.  However, these results are not guaranteed, and if there was a gap of more than 3-5% between in-sample + out-of-sample, we might try and tweak these to improve our results.

However, to improve fitting times, let's see if we can tweak some GBM parameters.

In [49]:
# we'll try a GBM that only uses 70% of columns at each split
pipe = make_pipeline(ce.TargetEncoder(), GradientBoostingRegressor(max_features=0.7))

# and see our results
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_val, y_val)

(0.5220539599716651, 0.5525071701013251)

This seemed like it had the intended effect -- faster fitting times, no appreciable change in scores, so we'll keep it.

**Step 3:** Using the results that gave you the best answer from above, try now to find model parameters that maximize information extraction.  The three main ones are:

 - `n_estimators`:  how many boosting rounds to use
 - `learning_rate`: how much shrinkage to use at each update (keep this from .05 to .2)
 - `max_depth`: how deep each tree in your model goes
 
 **important:** fitting these things could take a looooong time.  We don't have all night.  So don't try and make this exhaustive, just try doing a little bit of parameter exploration to see if you can see in what directions to push model parameters to improve your results.  
 
 Note your validation score before proceeding to the next step.

In [53]:
# your code here
# setup model parameters
n_estimators  = [100, 250, 500]
learning_rate = [.05, .1, .2]
max_depth     = [3, 4, 5, 6]
cv_scores     = []

# and cycle through our model parameters
for estimators in n_estimators:
    for rate in learning_rate:
        for depth in max_depth:
            print(f"Fitting model with parameters:  n_estimators - {estimators}, learning_rate - {rate}, max_depth - {depth}")
            mod   = GradientBoostingRegressor(n_estimators=estimators, learning_rate=rate, max_depth=depth, max_features=0.6)
            pipe  = make_pipeline(ce.TargetEncoder(), mod)
            pipe.fit(X_train, y_train)
            score = pipe.score(X_val, y_val)
            print(f"Out-of-sample score: {score}")
            cv_scores.append((score, estimators, rate, depth))

Fitting model with parameters:  n_estimators - 100, learning_rate - 0.05, max_depth - 3
Out-of-sample score: 0.5440895818411422
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.05, max_depth - 4
Out-of-sample score: 0.5529674736267103
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.05, max_depth - 5
Out-of-sample score: 0.5617630217802974
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.05, max_depth - 6
Out-of-sample score: 0.571682201805663
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.1, max_depth - 3
Out-of-sample score: 0.5528157678655643
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.1, max_depth - 4
Out-of-sample score: 0.5637513461215058
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.1, max_depth - 5
Out-of-sample score: 0.5706105603270158
Fitting model with parameters:  n_estimators - 100, learning_rate - 0.1, max_depth - 6
Out-of-sample score: 

**Step 4:** Take the best version of your model & your data, and fit it on **all** of your training + validation data.  The idea is that now that we've found the best version of what we have to work with, we want to give it as much training samples as possible.  

In [68]:
# your code here
pipe[1].set_params(n_estimators=500, max_depth=6, learning_rate=.1)
train, test = create_val_splits(df)

X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_test, y_test   = test.drop('visitors', axis=1), test['visitors']

pipe.fit(X_train, np.log(y_train))

[17:47:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'genre', 'area',
                                     'city'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', min_samples_leaf=1,
                               return_df=True, smoothing=1.0, verbose=0)),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0,
                              importance_type='gain', learning_rate=0.1,
                              max_delta_step=0, max_depth=6, max_features=0.6,
                              min_child_weight=1, missing=None,
                              n_estimators=500, n_jobs=1, nthread=None,
                              objective='reg:linear', random_state=0,
           

**Step 5:** Score your model on your test set.

Note how your validation + test scores compared to one another.

In [71]:
pipe.score(X_test, y_test)

0.6074617988944077

In [42]:
y = 80
y_hat = 60

In [46]:
(np.log(y) - np.log(y_hat))**2

0.08276097481015166

In [44]:
y = 8
y_hat = 6

In [47]:
(np.log(y) - np.log(y_hat))**2

0.08276097481015166

So in this case, we would report our results with a validation score of 0.589 + test scoe of 0.56.  Modest overfitting, but not too extreme.  Subsequent iterations could provide a bit more smoothing to even them out.